In [1]:
import xarray as xr
import pandas as pd
import plotly.express as px

## Import data


In [ ]:
station = "Papa"

In [3]:
obs_data = xr.open_dataset(f"../data/1_products/{station}_obs_zoo_climato_monthly_2002_2015.zarr", engine="zarr")

In [4]:
cafe_data = xr.open_dataset(f"../data/1_products/{station}_cafe.zarr", engine="zarr")
cafe_data = cafe_data.resample(time="1D").mean(dim="time").interpolate_na(dim="time", method="linear").CAFE
first_year = str(obs_data.indexes["time"].to_pydatetime()[0].year)
last_year = str(obs_data.indexes["time"].to_pydatetime()[-1].year)
cafe_data = cafe_data.sel(time=slice(first_year, last_year))
cafe_data

<xarray.DataArray 'CAFE' (time: 4748, latitude: 1, longitude: 1)> Size: 38kB
array([[[243.08382813]],

       [[245.94637508]],

       [[248.80892204]],

       ...,

       [[245.7222222 ]],

       [[250.56580957]],

       [[255.40939694]]])
Coordinates:
  * latitude   (latitude) float64 8B 31.6
  * longitude  (longitude) float64 8B -64.2
  * time       (time) datetime64[ns] 38kB 2002-01-01 2002-01-02 ... 2014-12-31
Attributes: (12/13)
    adg wavelength:               443nm
    aph wavelength:               443nm
    bbp wavelength:               443nm
    doi:                          10.1002/2016GB005521
    long_name:                    primary_production_using_CAFE_model
    mld_criteria:                 0.125 kg m^-3
    ...                           ...
    reference:                    Silsbe_et_al_2016
    remote_sensing_data_product:  OCCCIv6
    resolution:                   0.00f; // float
    standard_name:                NPP
    units:                        mg m-2 day-1
    weight:                       carbon

## Create climatology


In [5]:
cafe_data_climato = cafe_data.groupby("time.dayofyear").mean()

all_years = []
for year in range(int(cafe_data.time.min().dt.year), int(cafe_data.time.max().dt.year + 1)):
    slice_max = 366
    if not pd.Timestamp(f"{year}-12-31").is_leap_year:
        slice_max = 365

    yearly_data = cafe_data_climato.sel(dayofyear=slice(1, slice_max)).assign_coords(
        time=(
            "dayofyear",
            pd.date_range(
                start=f"{year}-01-01",
                end=f"{year}-12-31",
                freq="D",
                name="time",
            ),
        ),
    )

    all_years.append(yearly_data.swap_dims({"dayofyear": "time"}))
cafe_data_climato_expanded = xr.concat(all_years, dim="time").drop_vars("dayofyear")
cafe_data_climato_expanded.time.attrs = cafe_data.time.attrs
cafe_data_climato_expanded.attrs["description"] = "CAFE climatology (2002-2015) interpolated to daily frequency"
cafe_data_climato_expanded

<xarray.DataArray 'CAFE' (time: 4748, latitude: 1, longitude: 1)> Size: 38kB
array([[[219.87516943]],

       [[222.91282933]],

       [[225.95048922]],

       ...,

       [[219.20559275]],

       [[219.57000319]],

       [[219.93441362]]])
Coordinates:
  * latitude   (latitude) float64 8B 31.6
  * longitude  (longitude) float64 8B -64.2
  * time       (time) datetime64[ns] 38kB 2002-01-01 2002-01-02 ... 2014-12-31
Attributes: (12/14)
    adg wavelength:               443nm
    aph wavelength:               443nm
    bbp wavelength:               443nm
    doi:                          10.1002/2016GB005521
    long_name:                    primary_production_using_CAFE_model
    mld_criteria:                 0.125 kg m^-3
    ...                           ...
    remote_sensing_data_product:  OCCCIv6
    resolution:                   0.00f; // float
    standard_name:                NPP
    units:                        mg m-2 day-1
    weight:                       carbon
    description:                  CAFE climatology (2002-2015) interpolated t...

## Plot


In [6]:
table_cafe = (
    pd.DataFrame(
        {"CAFE": cafe_data.to_dataframe()["CAFE"], "CAFE_climato": cafe_data_climato_expanded.to_dataframe()["CAFE"]}
    )
    .stack()
    .droplevel("latitude")
    .droplevel("longitude")
    .reset_index()
    .rename(columns={"level_1": "type", 0: "NPP"})
)

In [7]:
px.line(table_cafe, x="time", y="NPP", color="type", title=f"CAFE vs CAFE_climato : {station}")

## Export


In [ ]:
cafe_data_climato_expanded.to_zarr(f"../data/1_products/{station}_cafe_climato.zarr", mode="w")